# Visualizing and reducing WFM imaging data with Scipp


## How to start

Before starting you must:

- Have conda installed
- `conda env create -f ess-notebooks-latest.yml python=3.7` . The yaml environment file is part of this repository.
- fetch the data `git clone git@github.com:scipp/ess-notebooks-data.git` somewhere local 
- Generate the `dataconfig.py` file using `make_config.py` located in same directory as this notebook. In general, you simply need to point `make_config.py` to the root directory of data you cloned above. Refer to the help `make_config.py --help` for more information. 

## Experimental Summary

This script has been developed to measure local strain $\varepsilon$ defined as $\varepsilon = \Delta L/L_{0}$ in a FCC steel sample under elastic strain in a stress rig.
The measurements were measured at V20, HZB, Berlin, in September 2018 by Peter Kadletz.

$\lambda = 2 d \sin\theta$, where $2\theta = \pi$ (transmission), edges characterise the Bragg condition and hence $\lambda = 2 d$.
Therefore strain is easily computed from the wavelength measurement of a Bragg edge directly, using un-loaded vs loaded experimental runs (and reference mesurements).

The known Miller indices of the crystal structure (FCC) are used to predict the wavelength of the Bragg edges, which is bound by the reachable wavelength extents of the instrument.
This provides an approximate region to apply a fit.
A complement error function is used to fit each Bragg edge, and a refined centre location ($\lambda$) for the edge is used in the strain measurement.
Because each Bragg edge can be identified individually, one can determine anisotropic strain across the unit cell in the reachable crystallographic directions.

<img src="IMG_2290.jpg" width="500" />

# Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import scipp as sc
import scippneutron as sn
from scipp import plot

import os
import sys

import ess.v20.imaging as imaging
import ess.v20.imaging.operations as operations
import ess.v20.wfm as wfm
from ess.v20.wfm.beamline import choppers

import dataconfig

local_data_path = os.path.join('ess', 'v20', 'imaging',
                               'gp2-stress-experiments')
data_dir = os.path.join(dataconfig.data_root, local_data_path)
instrument_file = os.path.join(data_dir, 'V20_Definition_GP2.xml')
tofs_path = os.path.join(data_dir, 'GP2_Stress_time_values.txt')
raw_data_dir = os.path.join(data_dir)

# Reduction

## Load the data files and instrument geometry

In [ ]:
def load_and_scale(folder_name, scale_factor):
    to_load = os.path.join(raw_data_dir, folder_name)
    variable = imaging.tiffs_to_variable(to_load, dtype=np.float32, with_variances=False)
    variable *= scale_factor
    return variable

In [ ]:
# Number of pulses for each run, to scale data according to integration times
pulse_number_reference = 1.0 / 770956
pulse_number_sample = 1.0 / 1280381
pulse_number_sample_elastic = 1.0 / 2416839
pulse_number_sample_plastic = 1.0 / 2614343

# Create Dataset
ds = sc.Dataset()

# Load time bins from 1D text file
ds.coords["t"] = sc.Variable(
    ["t"],
    unit=sc.units.us,
    values=imaging.read_x_values(tofs_path,
                                 skiprows=1,
                                 usecols=1,
                                 delimiter='\t') * 1.0e3)

# Load tiff stack
ds["reference"] = load_and_scale(folder_name="R825-open-beam",
                                 scale_factor=pulse_number_reference)
ds["sample"] = load_and_scale(folder_name="R825",
                              scale_factor=pulse_number_sample)
ds["sample_elastic"] = load_and_scale(
    folder_name="R825-600-Mpa", scale_factor=pulse_number_sample_elastic)

# Instrument geometry
geometry = sc.Dataset()
sn.mantid.load_component_info(geometry, instrument_file)
geom = sc.Dataset(coords={"source_position": geometry.coords["source_position"]})
geom.coords["position"] = sc.reshape(geometry.coords['position'],
                                     dims=['y', 'x'],
                                     shape=tuple(ds["sample"]["t",
                                                              0].shape))
geom.coords["x"] = sc.geometry.x(geom.coords["position"])["y", 0]
geom.coords["y"] = sc.geometry.y(geom.coords["position"])["x", 0]
ds = sc.merge(ds, geom)

In [ ]:
ds

## Raw data visualization

In [ ]:
plot(ds["sample"])

## Converting time coordinate to TOF

Use the instrument geometry and chopper cascade parameters to compute time-distance diagram.

In [ ]:
beamline = choppers()
# Add the detector pixel positions
beamline["position"] = ds.coords["position"] - ds.coords[
    "source_position"] + sc.scalar(0.5 * np.sum(beamline["distance"]["chopper", 0:2].values, axis=0),
                 dtype=sc.dtype.vector_3_float64, unit=beamline["distance"].unit)
beamline

In [ ]:
frames = wfm.get_frames(instrument=beamline, plot=True)
frames

Assume all pixels have the same frame boundaries, so we take the mean boundaries for each frame (this saves memory in the stitching step).

In [ ]:
for key in ["left_edges", "right_edges"]:
    frames[key] = sc.mean(sc.mean(frames[key], 'x'), 'y')
frames

We overlay the frame locations onto a spectrum of integrated counts

In [ ]:
fig1, ax1 = plt.subplots()
plot(sc.sum(sc.sum(ds["reference"], 'x'), 'y'), ax=ax1)
for i in range(frames.sizes["frame"]):
    ax1.axvspan(frames["left_edges"]["frame", i].value,
                frames["right_edges"]["frame", i].value, color="C{}".format(i), alpha=0.3)

Extract the sections in the original data using value-based slicing and shift the coordinates:

In [ ]:
stitched = wfm.stitch(data=ds, dim="t", frames=frames, plot=True)
stitched

To stitch the data, we make a common container with a TOF axis spanning the entire range, and sum the counts from the different frames.

In [ ]:
plot(sc.sum(sc.sum(stitched, 'x'), 'y'))

## Crop to relevant Tof section

We take a subset of the Tof range which contains the Bragg edges of interest:

In [ ]:
tof_start = 9.0e3*sc.units.us
tof_end = 2.75e4*sc.units.us
stitched = stitched["tof", tof_start:tof_end].copy()
plot(sc.sum(sc.sum(stitched, 'x'), 'y'))

## Transmission Masking

Divides the integrated sample counts with an open beam reference. Any values > masking threshold will be masked. The adj pixels step checks for random pixels which were left unmasked or masked with all their neighbours having the same mask value. These are forced to True or false depending on their neighbour value.

In [ ]:
integrated = sc.sum(stitched, 'tof')
integrated /= integrated["reference"]

In [ ]:
masking_threshold = 0.80 * sc.units.one

for key in ["sample", "sample_elastic"]:
    mask = integrated[key].data > masking_threshold
    # Apply some neighbour smoothing to the masks
    mask = operations.mask_from_adj_pixels(mask)
    stitched[key].masks["non-sample-region"] = mask
stitched

In [ ]:
plot(sc.sum(stitched["sample"], "tof"))

## Convert to wavelength

Scipp's `neutron` submodule contains utilities specific to neutron science, and in particular unit conversions.

In [ ]:
wavelength = sn.convert(stitched, origin="tof", target="wavelength", scatter=False)

# Rebin to common wavelength axis
edges = sc.array(dims=["wavelength"],
                       values=np.linspace(2.0, 5.5, 129), unit=sc.units.angstrom)
wavelength = sc.rebin(wavelength, "wavelength", edges)

wavelength

## Apply mean filter

In [ ]:
for key in wavelength:
    wavelength[key].data = operations.mean_from_adj_pixels(wavelength[key].data)
plot(wavelength["sample"])

## Other visualizations

### Show difference between sample and elastic

In [ ]:
plot(wavelength["sample"] - wavelength["sample_elastic"], vmin=-0.002, vmax=0.002, cmap="RdBu")

## Group detector pixels to increase signal to noise

In [ ]:
nbins=27
grouped = operations.groupby2D(wavelength, nx_target=nbins, ny_target=nbins)
for key, item in grouped.items():
    item.masks["zero-counts"] = item.data == 0.0*sc.units.counts

In [ ]:
plot(grouped["sample"])

## Normalization

In [ ]:
# Normalize by open beam
normalized = grouped / grouped["reference"]
del normalized["reference"]

In [ ]:
replacement = sc.Variable(value=0.0, variance=0.0)
kwargs = {"nan": replacement, "posinf": replacement, "neginf": replacement}
for k in normalized.keys():
    sc.nan_to_num(normalized[k].data, out=normalized[k].data, **kwargs)

summed = sc.sum(sc.sum(normalized, 'x'), 'y')    
plot(summed, grid=True, title='I/I0')

# Bragg edge fitting

We will first carry out Bragg-edge fitting on the sum of all counts in the sample.

## Calculate expected Bragg edge positions 

In [ ]:
def create_Braggedge_list(lattice_constant, miller_indices):
    '''
    :param miller-indices: like [(1,1,0),(2,0,0),...]
    :type miller-indices: list of tuples
    '''
    coords = [str((h, k, l)) for h, k, l in miller_indices]
    interplanar_distances = [
        2. * lattice_constant / np.sqrt(h**2 + k**2 + l**2)
        for h, k, l in miller_indices
    ]

    d = sc.DataArray(
        sc.Variable(dims=["bragg-edge"],
                    values=np.array(interplanar_distances),
                    unit=sc.units.angstrom))
    d.coords["miller-indices"] = sc.Variable(dims=["bragg-edge"],
                                             values=coords)
    return d

In [ ]:
# Bragg edge position, in Angstrom, taken from COD entry 9008469
FCC_a = 3.5

# These miller indices for the given unit cell yield bragg edges
# between the maximum and minimum wavelength range.
indices_FCC = [(1, 1, 1), (2, 0, 0), (2, 2, 0), (3, 1, 1)]

Bragg_edges_FCC = create_Braggedge_list(FCC_a, indices_FCC)
sc.table(Bragg_edges_FCC)

## Run the fitting (currently using Mantid under the hood)

In [ ]:
def fit(Bragg_edges_FCC, reference, quiet=False):
    """
    Fit a list of Bragg edges using Mantid.
    """

    x_min_sides = [0.05, 0.1, 0.1, 0.05]
    x_max_sides = [0.1, 0.05, 0.1, 0.1]
    fit_list = []

    for edge_index in range(Bragg_edges_FCC.shape[0]):

        bragg_edge = Bragg_edges_FCC.coords["miller-indices"]["bragg-edge",
                                                              edge_index]

        # Initial guess
        xpos_guess = Bragg_edges_FCC["bragg-edge", edge_index].value
        x_min_fit = xpos_guess - xpos_guess * abs(x_min_sides[edge_index])
        x_max_fit = xpos_guess + xpos_guess * abs(x_max_sides[edge_index])

        if not quiet:
            print(
                "Now fitting Bragg edge {} at {:.3f} A (between {:.3f} A and {:.3f} A) across image groups"
                .format(bragg_edge, xpos_guess, x_min_fit, x_max_fit))

        # Call Mantid fitting
        params_s, diff_s = sn.mantid.fit(
            reference,
            mantid_args={
                'Function':
                f'name=LinearBackground,A0={270},A1={-10};name=UserFunction,Formula=h*erf(a*(x-x0)),h={200},a={-11},x0={xpos_guess}',
                'StartX': x_min_fit,
                'EndX': x_max_fit
            })

        v_and_var_s = [
            params_s.data['parameter', i]
            for i in range(params_s['parameter', :].shape[0])
        ]
        params = dict(zip(params_s.coords["parameter"].values, v_and_var_s))

        fit_list.append(params)

    return fit_list

In [ ]:
fit_params = fit(Bragg_edges_FCC, reference=summed["sample"])

In [ ]:
import scipy as sp
fig, ax = plt.subplots()
summed["sample"].plot(ax=ax)
for i, f in enumerate(fit_params):
    x = np.linspace(f['f1.x0'].value - 0.3, f['f1.x0'].value + 0.3, 100)
    y=f['f1.h'].value*sp.special.erf(f['f1.a'].value*(x-f['f1.x0'].value)) + (f['f0.A0'].value + f['f0.A1'].value*x)
    ax.plot(x, y, color="C{}".format(i+1), label=Bragg_edges_FCC.coords['miller-indices'].values[i])
ax.legend()

## Create a strain map

We can create a strain map by fitting the Bragg edges inside each tile of the image.

In [ ]:
# Make an empty container to receive the strain values
strain_map = normalized["sample_elastic"]["wavelength", 0] * (0.0 * sc.units.one)

iprog = 0
step = 5

# Loop over all the tiles
for j in range(nbins):
    for i in range(nbins):

        prog = int(100 * (i + j*nbins) / (nbins*nbins))
        if prog % step == 0 and prog == iprog:
            print(prog, "% complete")
            iprog += step
        
        if sc.greater(sc.sum(normalized["sample_elastic"]['y', j]['x', i], "wavelength").data,
                      0.0*sc.units.one).value:
            elastic_fit = fit(Bragg_edges_FCC,
                                    reference=normalized["sample_elastic"]['y', j]['x', i], quiet=True)
            strain_map['y', j]['x', i] = 0.5 * (elastic_fit[0]['f1.x0'] - fit_params[0]['f1.x0'])

In [ ]:
plot(strain_map, vmin=-0.02, vmax=0.02, cmap="RdBu", title="Lattice strain $\epsilon$ (1, 1, 1)")